<a href="https://colab.research.google.com/github/talhagedikli/ml-projects/blob/main/19069608_Code_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To obtain shipment schedule

## Construct the mathematical model

```
x[i][j]: The number of books sent from warehouse i to distribution center j.
y[j][k]: The number of books sent from distribution center j to store k.
z[i][k]: The number of books sent directly from warehouse i to store k.
```

The objective is to minimize the total cost of transportation, which can be expressed as:

`minimize sum(c[i][j]*x[i][j] for i in warehouses for j in distribution_centers) + sum(d[j][k]*y[j][k] for j in distribution_centers for k in stores) + sum(e[i][k]*z[i][k] for i in warehouses for k in stores)`

where `c[i][j]` is the cost of sending one book from warehouse `i` to distribution center `j`, `d[j][k]` is the cost of sending one book from distribution center `j` to store k, and `e[i][k]` is the cost of sending one book directly from warehouse `i` to store `k`.
The constraints are:
The total number of books sent from each warehouse must not exceed its capacity:

`sum(x[i][j] for j in distribution_centers) + sum(z[i][k] for k in stores) <= capacity[i] for i in warehouses`

The total number of books received by each store must meet its demand:

`sum(y[j][k] for j in distribution_centers) + sum(z[i][k] for i in warehouses) == demand[k] for k in stores`

The total number of books sent from each distribution center must equal the total number received:

`sum(x[i][j] for i in warehouses) == sum(y[j][k] for k in stores) for j in distribution_centers`

All variables must be non-negative:
```
x[i][j] >= 0 for i in warehouses for j in distribution_centers
y[j][k] >= 0 for j in distribution_centers for k in stores
z[i][k] >= 0 for i in warehouses for k in stores
```


## Create your own shipment cost values by using Python code given in jupyter notebook

In [ ]:
import numpy as np
import pandas as pd
import math

'''
Important note!!
I randomly generated numbers here in a different way. Numbers like 0, 1, 2 (like in x[i][j])
represent things like Tonoz(0), White_Kiosk(1), Quarterage(2). I explained these clearly in the results.

Here is a quick explanation:

x[i][j]: The number of books sent from warehouse i to distribution center j.
y[j][k]: The number of books sent from distribution center j to store k.
z[i][k]: The number of books sent directly from warehouse i to store k.

Warehouses (i):             [Tonoz(0), White_Kiosk(1), Quarterage(2), Middle_Yard(3)]
Distribution Centers (j):   [Atlas(0), Nebulae(1), Azure(2)]
Stores (k):                 [Venus(0), Minerva(1), Neptunus(2), Mars(3), Ceres(4)]
'''


# Number of warehouses
n_warehouses = 4

# Number of distribution centers
n_distribution_centers = 3

# Number of stores
n_stores = 5

np.random.seed(19069608)

# Generate random cost values for sending books from warehouses to distribution centers
c = np.random.randint(1, 10, size=(n_warehouses, n_distribution_centers))

# Generate random cost values for sending books from distribution centers to stores
d = np.random.randint(1, 10, size=(n_distribution_centers, n_stores))

# Generate random cost values for sending books directly from warehouses to stores
e = np.random.randint(1, 10, size=(n_warehouses, n_stores))

print("The shipment cost from factory to distribution center")
print(c)
print("The shipment cost from distribution center to store")
print(d)
print("The shipment cost from factory to store")
print(e)

The shipment cost from factory to distribution center
[[2 8 8]
 [7 2 3]
 [3 2 1]
 [6 8 9]]
The shipment cost from distribution center to store
[[6 9 6 3 6]
 [4 6 3 4 9]
 [4 6 1 8 9]]
The shipment cost from factory to store
[[9 5 9 7 7]
 [3 6 3 3 9]
 [3 6 8 6 6]
 [7 5 4 4 9]]


This code generates random cost values for sending books from warehouses to distribution centers (c), from distribution centers to stores (d), and directly from warehouses to stores (e). The cost values are integers between 1 and 9 (inclusive).

## Code your mathematical model with parameters

In [ ]:
# I will use pulp to create and solve the problem
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum
import numpy as np

# Define the problem data
n_warehouses = 4
n_distribution_centers = 3
n_stores = 5

capacity = [3000, 5000, 10000, 7000]
demand = [5500, 4750, 6550, 4000, 4200]

np.random.seed(19069608)
c = np.random.randint(1, 10, size=(n_warehouses, n_distribution_centers))
d = np.random.randint(1, 10, size=(n_distribution_centers, n_stores))
e = np.random.randint(1, 10, size=(n_warehouses, n_stores))


# Define the decision variables
x = LpVariable.dicts("x", (range(n_warehouses), range(n_distribution_centers)), lowBound=0)
y = LpVariable.dicts("y", (range(n_distribution_centers), range(n_stores)), lowBound=0)
z = LpVariable.dicts("z", (range(n_warehouses), range(n_stores)), lowBound=0)

# Define the problem
prob = LpProblem("TransportationProblem", LpMinimize)

# Define the objective function
cost = lpSum(c[i][j]*x[i][j] for i in range(n_warehouses) for j in range(n_distribution_centers)) + lpSum(d[j][k]*y[j][k] for j in range(n_distribution_centers) for k in range(n_stores)) + lpSum(e[i][k]*z[i][k] for i in range(n_warehouses) for k in range(n_stores))
prob += cost

# Define the constraints
# The total number of books sent from each warehouse must not exceed its capacity
for i in range(n_warehouses):
    prob += lpSum(x[i][j] for j in range(n_distribution_centers)) + lpSum(z[i][k] for k in range(n_stores)) <= capacity[i]

# The total number of books received by each store must meet its demand
for k in range(n_stores):
    prob += lpSum(y[j][k] for j in range(n_distribution_centers)) + lpSum(z[i][k] for i in range(n_warehouses)) == demand[k]

# The total number of books sent from each distribution center must equal the total number received
for j in range(n_distribution_centers):
    prob += lpSum(x[i][j] for i in range(n_warehouses)) == lpSum(y[j][k] for k in range(n_stores))


This code defines a linear programming problem using the pulp library. The decision variables are x, y, and z, which represent the number of books sent from warehouses to distribution centers (x), from distribution centers to stores (y), and directly from warehouses to stores (z). The objective is to minimize the total cost of transportation (cost), subject to constraints on the capacity of warehouses and the demand of stores.

The problem is solved using the solve method of the LpProblem object and the open-source CBC solver. The optimal solution is printed at the end.

## Solve the model

In [ ]:
# Solve the problem using the CBC solver
prob.solve()

# Print the optimal solution
print("Optimal transportation plan:")
for i in range(n_warehouses):
    for j in range(n_distribution_centers):
      # Do not print the value if it's 0 value (means no distribution between them)
      if (x[i][j].value() > 0):
        print(f"x[{i}][{j}] =", x[i][j].value())
for j in range(n_distribution_centers):
    for k in range(n_stores):
      # Do not print the value if it's 0 value (means no distribution between them)
      if (y[j][k].value() > 0):
        print(f"y[{j}][{k}] =", y[j][k].value())
for i in range(n_warehouses):
    for k in range(n_stores):
      # Do not print the value if it's 0 value (means no distribution between them)
      if (z[i][k].value() > 0):
        print(f"z[{i}][{k}] =", z[i][k].value())
print('------------------------------------------------------------------------')
print("Total (optimal minimum) cost:", cost.value())
print('------------------------------------------------------------------------')
print('x[i][j]: The number of books sent from \x1B[3mwarehouse\x1B[0m i to \x1B[3mdistribution center\x1B[0m j.')
print('y[j][k]: The number of books sent from \x1B[3mdistribution center\x1B[0m j to \x1B[3mstore\x1B[0m k.')
print('z[i][k]: The number of books sent directly from \x1B[3mwarehouse\x1B[0m i to \x1B[3mstore\x1B[0m k.')
print('------------------------------------------------------------------------')
print("Warehouses (i):              [Tonoz(0), White_Kiosk(1), Quarterage(2), Middle_Yard(3)]")
print("Distribution Centers (j):    [Atlas(0), Nebulae(1), Azure(2)]")
print("Stores (k):                  [Venus(0), Minerva(1), Neptunus(2), Mars(3), Ceres(4)]")

Optimal transportation plan:
x[2][2] = 6550.0
y[2][2] = 6550.0
z[0][1] = 1750.0
z[0][4] = 1250.0
z[1][0] = 5000.0
z[2][0] = 500.0
z[2][4] = 2950.0
z[3][1] = 3000.0
z[3][3] = 4000.0
------------------------------------------------------------------------
Total (optimal minimum) cost: 95800.0
------------------------------------------------------------------------
x[i][j]: The number of books sent from warehouse i to distribution center j.
y[j][k]: The number of books sent from distribution center j to store k.
z[i][k]: The number of books sent directly from warehouse i to store k.
------------------------------------------------------------------------
Warehouses (i):              [Tonoz(0), White_Kiosk(1), Quarterage(2), Middle_Yard(3)]
Distribution Centers (j):    [Atlas(0), Nebulae(1), Azure(2)]
Stores (k):                  [Venus(0), Minerva(1), Neptunus(2), Mars(3), Ceres(4)]


# Questions

## a) What is your mathematical model?

The decision variables in the model are:

    x[i][j]: The number of books sent from warehouse i to distribution center j.
    y[j][k]: The number of books sent from distribution center j to store k.
    z[i][k]: The number of books sent directly from warehouse i to store k.

The objective function is to minimize the total cost of transportation, which is calculated as the sum of the products of the number of books transported along each route and the cost per book for that route.

The constraints in the model ensure that:

    The total number of books sent from each warehouse does not exceed its capacity.
    The total number of books received by each store meets its demand.
    The total number of books sent from each distribution center equals the total number received.


## b) What is value of the objective function?

In [ ]:
# The value of the objective function is printed using the statement
print("Total cost =", cost.value())

Total cost = 95800.0


## c) What is the value of decision variables? According to this values, evaluate the supply chain and flow in this supply chain?

In [ ]:
# Define a function for readability
def print_solution_and_cost():
  print("Optimal transportation plan:")
  for i in range(n_warehouses):
      for j in range(n_distribution_centers):
        # Do not print the value if it's 0 value (means no distribution between them)
        if (x[i][j].value() > 0):
          print(f"x[{i}][{j}] =", x[i][j].value())
  for j in range(n_distribution_centers):
      for k in range(n_stores):
        # Do not print the value if it's 0 value (means no distribution between them)
        if (y[j][k].value() > 0):
          print(f"y[{j}][{k}] =", y[j][k].value())
  for i in range(n_warehouses):
      for k in range(n_stores):
        # Do not print the value if it's 0 value (means no distribution between them)
        if (z[i][k].value() > 0):
          print(f"z[{i}][{k}] =", z[i][k].value())
  print('------------------------------------------------------------------------')
  print("Total (optimal minimum) cost:", cost.value())
  print('------------------------------------------------------------------------')
  print('x[i][j]: The number of books sent from \x1B[3mwarehouse\x1B[0m i to \x1B[3mdistribution center\x1B[0m j.')
  print('y[j][k]: The number of books sent from \x1B[3mdistribution center\x1B[0m j to \x1B[3mstore\x1B[0m k.')
  print('z[i][k]: The number of books sent directly from \x1B[3mwarehouse\x1B[0m i to \x1B[3mstore\x1B[0m k.')
  print('------------------------------------------------------------------------')
  print("Warehouses (i):              [Tonoz(0), White_Kiosk(1), Quarterage(2), Middle_Yard(3)]")
  print("Distribution Centers (j):    [Atlas(0), Nebulae(1), Azure(2)]")
  print("Stores (k):                  [Venus(0), Minerva(1), Neptunus(2), Mars(3), Ceres(4)]")



# Print the optimal solution
print_solution_and_cost()

Optimal transportation plan:
x[2][2] = 6550.0
y[2][2] = 6550.0
z[0][1] = 1750.0
z[0][4] = 1250.0
z[1][0] = 5000.0
z[2][0] = 500.0
z[2][4] = 2950.0
z[3][1] = 3000.0
z[3][3] = 4000.0
------------------------------------------------------------------------
Total (optimal minimum) cost: 95800.0
------------------------------------------------------------------------
x[i][j]: The number of books sent from warehouse i to distribution center j.
y[j][k]: The number of books sent from distribution center j to store k.
z[i][k]: The number of books sent directly from warehouse i to store k.
------------------------------------------------------------------------
Warehouses (i):              [Tonoz(0), White_Kiosk(1), Quarterage(2), Middle_Yard(3)]
Distribution Centers (j):    [Atlas(0), Nebulae(1), Azure(2)]
Stores (k):                  [Venus(0), Minerva(1), Neptunus(2), Mars(3), Ceres(4)]


The values of the decision variables represent the number of books sent along each route in the transportation network. For example, `x[i][j]` represents the number of books sent from warehouse `i` to distribution center `j`, `y[j][k]` represents the number of books sent from distribution center `j` to store `k`, and `z[i][k]` represents the number of books sent directly from warehouse `i` to store `k`. By analyzing these values, we can evaluate the flow of books in the supply chain and see how many books are sent along each route.

## d) If we change the capacity of warehouse Quarterage from 10000 to 7500, and the demand of Ceres from 4200 to 1700, how do the objective value and the values of the decision variables change?

If we change the capacity of warehouse Quarterage from 10000 to 7500, and the demand of Ceres from 4200 to 1700, the values of the decision variables and the objective function will change.

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum
import numpy as np

# Define the problem data
n_warehouses = 4
n_distribution_centers = 3
n_stores = 5

capacity = [3000, 5000, 7500, 7000] # changed capacity of Quarterage from 10000 to 7500
demand = [5500, 4750, 6550, 4000, 1700] # changed demand of Ceres from 4200 to 1700

np.random.seed(19069608)
c = np.random.randint(1, 10, size=(n_warehouses, n_distribution_centers))
d = np.random.randint(1, 10, size=(n_distribution_centers, n_stores))
e = np.random.randint(1, 10, size=(n_warehouses, n_stores))

# Define the decision variables
x = LpVariable.dicts("x", (range(n_warehouses), range(n_distribution_centers)), lowBound=0)
y = LpVariable.dicts("y", (range(n_distribution_centers), range(n_stores)), lowBound=0)
z = LpVariable.dicts("z", (range(n_warehouses), range(n_stores)), lowBound=0)

# Define the problem
prob = LpProblem("TransportationProblem", LpMinimize)

# Define the objective function
cost = lpSum(c[i][j]*x[i][j] for i in range(n_warehouses) for j in range(n_distribution_centers)) + lpSum(d[j][k]*y[j][k] for j in range(n_distribution_centers) for k in range(n_stores)) + lpSum(e[i][k]*z[i][k] for i in range(n_warehouses) for k in range(n_stores))
prob += cost

# Define the constraints
# The total number of books sent from each warehouse must not exceed its capacity
for i in range(n_warehouses):
    prob += lpSum(x[i][j] for j in range(n_distribution_centers)) + lpSum(z[i][k] for k in range(n_stores)) <= capacity[i]

# The total number of books received by each store must meet its demand
for k in range(n_stores):
    prob += lpSum(y[j][k] for j in range(n_distribution_centers)) + lpSum(z[i][k] for i in range(n_warehouses)) == demand[k]

# The total number of books sent from each distribution center must equal the total number received
for j in range(n_distribution_centers):
    prob += lpSum(x[i][j] for i in range(n_warehouses)) == lpSum(y[j][k] for k in range(n_stores))

# Solve the problem using
prob.solve()

# Print the optimal solution
print_solution_and_cost()

Optimal transportation plan:
x[2][2] = 6550.0
y[2][2] = 6550.0
z[0][1] = 1750.0
z[0][4] = 1250.0
z[1][0] = 5000.0
z[2][0] = 500.0
z[2][4] = 450.0
z[3][1] = 3000.0
z[3][3] = 4000.0
------------------------------------------------------------------------
Total (optimal minimum) cost: 80800.0
------------------------------------------------------------------------
x[i][j]: The number of books sent from warehouse i to distribution center j.
y[j][k]: The number of books sent from distribution center j to store k.
z[i][k]: The number of books sent directly from warehouse i to store k.
------------------------------------------------------------------------
Warehouses (i):              [Tonoz(0), White_Kiosk(1), Quarterage(2), Middle_Yard(3)]
Distribution Centers (j):    [Atlas(0), Nebulae(1), Azure(2)]
Stores (k):                  [Venus(0), Minerva(1), Neptunus(2), Mars(3), Ceres(4)]


## e) If we change the shipment cost from warehouse Tonoz to distribution center Atlas as 10 and Azure to Minerva as 5, how the objective function is affected. Give new objective value and number of shipments, and then explain your answer with few sentences.

If we change the shipment cost from warehouse Tonoz to distribution center Atlas to 10 and from distribution center Azure to store Minerva to 5, the values of the decision variables and the objective function will change.

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum
import numpy as np

# Define the problem data
n_warehouses = 4
n_distribution_centers = 3
n_stores = 5

capacity = [3000, 5000, 10000, 7000]
demand = [5500, 4750, 6550, 4000, 4200]

np.random.seed(19069608)
c = np.random.randint(1, 10, size=(n_warehouses, n_distribution_centers))
d = np.random.randint(1, 10, size=(n_distribution_centers, n_stores))
e = np.random.randint(1, 10, size=(n_warehouses, n_stores))

# Change shipment cost from warehouse Tonoz to distribution center Atlas to 10
c[0][0] = 10

# Change shipment cost from distribution center Azure to store Minerva to 5
d[2][1] = 5

# Define the decision variables
x = LpVariable.dicts("x", (range(n_warehouses), range(n_distribution_centers)), lowBound=0)
y = LpVariable.dicts("y", (range(n_distribution_centers), range(n_stores)), lowBound=0)
z = LpVariable.dicts("z", (range(n_warehouses), range(n_stores)), lowBound=0)

# Define the problem
prob = LpProblem("TransportationProblem", LpMinimize)

# Define the objective function
cost = lpSum(c[i][j]*x[i][j] for i in range(n_warehouses) for j in range(n_distribution_centers)) + lpSum(d[j][k]*y[j][k] for j in range(n_distribution_centers) for k in range(n_stores)) + lpSum(e[i][k]*z[i][k] for i in range(n_warehouses) for k in range(n_stores))
prob += cost

# Define the constraints
# The total number of books sent from each warehouse must not exceed its capacity
for i in range(n_warehouses):
    prob += lpSum(x[i][j] for j in range(n_distribution_centers)) + lpSum(z[i][k] for k in range(n_stores)) <= capacity[i]

# The total number of books received by each store must meet its demand
for k in range(n_stores):
    prob += lpSum(y[j][k] for j in range(n_distribution_centers)) + lpSum(z[i][k] for i in range(n_warehouses)) == demand[k]

# The total number of books sent from each distribution center must equal the total number received
for j in range(n_distribution_centers):
    prob += lpSum(x[i][j] for i in range(n_warehouses)) == lpSum(y[j][k] for k in range(n_stores))

# Solve the problem
prob.solve()

# Print the optimal solution
print_solution_and_cost()

Optimal transportation plan:
x[2][2] = 6550.0
y[2][2] = 6550.0
z[0][1] = 1750.0
z[0][4] = 1250.0
z[1][0] = 5000.0
z[2][0] = 500.0
z[2][4] = 2950.0
z[3][1] = 3000.0
z[3][3] = 4000.0
------------------------------------------------------------------------
Total (optimal minimum) cost: 95800.0
------------------------------------------------------------------------
x[i][j]: The number of books sent from warehouse i to distribution center j.
y[j][k]: The number of books sent from distribution center j to store k.
z[i][k]: The number of books sent directly from warehouse i to store k.
------------------------------------------------------------------------
Warehouses (i):              [Tonoz(0), White_Kiosk(1), Quarterage(2), Middle_Yard(3)]
Distribution Centers (j):    [Atlas(0), Nebulae(1), Azure(2)]
Stores (k):                  [Venus(0), Minerva(1), Neptunus(2), Mars(3), Ceres(4)]


## f) If we change the capacity of the warehouse Middle Yard from 7000 to 5000, what should we expect? Can we obtain optimal solution? Please explain your conclusion clearly. If it is necessary make some arrangement and give the results with explanations.

If we change the capacity of the warehouse Middle Yard from 7000 to 5000, the values of the decision variables and the objective function will change.

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum
import numpy as np

# Define the problem data
n_warehouses = 4
n_distribution_centers = 3
n_stores = 5

capacity = [3000, 5000, 10000, 5000] # changed capacity of Middle Yard from 7000 to 5000
demand = [5500, 4750, 6550, 4000, 4200]

np.random.seed(19069608)
c = np.random.randint(1, 10, size=(n_warehouses, n_distribution_centers))
d = np.random.randint(1, 10, size=(n_distribution_centers, n_stores))
e = np.random.randint(1, 10, size=(n_warehouses, n_stores))

# Define the decision variables
x = LpVariable.dicts("x", (range(n_warehouses), range(n_distribution_centers)), lowBound=0)
y = LpVariable.dicts("y", (range(n_distribution_centers), range(n_stores)), lowBound=0)
z = LpVariable.dicts("z", (range(n_warehouses), range(n_stores)), lowBound=0)

# Define the problem
prob = LpProblem("TransportationProblem", LpMinimize)

# Define the objective function
cost = lpSum(c[i][j]*x[i][j] for i in range(n_warehouses) for j in range(n_distribution_centers)) + lpSum(d[j][k]*y[j][k] for j in range(n_distribution_centers) for k in range(n_stores)) + lpSum(e[i][k]*z[i][k] for i in range(n_warehouses) for k in range(n_stores))
prob += cost

# Define the constraints
# The total number of books sent from each warehouse must not exceed its capacity
for i in range(n_warehouses):
    prob += lpSum(x[i][j] for j in range(n_distribution_centers)) + lpSum(z[i][k] for k in range(n_stores)) <= capacity[i]

# The total number of books received by each store must meet its demand
for k in range(n_stores):
    prob += lpSum(y[j][k] for j in range(n_distribution_centers)) + lpSum(z[i][k] for i in range(n_warehouses)) == demand[k]

# The total number of books sent from each distribution center must equal the total number received
for j in range(n_distribution_centers):
    prob += lpSum(x[i][j] for i in range(n_warehouses)) == lpSum(y[j][k] for k in range(n_stores))

# Solve the problem
prob.solve()

# Print the optimal solution
print_solution_and_cost()


Optimal transportation plan:
x[2][2] = 6550.0
y[2][2] = 6550.0
z[0][1] = 3000.0
z[1][0] = 6250.0
z[2][4] = 4200.0
z[3][1] = 1750.0
z[3][3] = 5250.0
------------------------------------------------------------------------
Total (optimal minimum) cost: 95800.0
------------------------------------------------------------------------
x[i][j]: The number of books sent from warehouse i to distribution center j.
y[j][k]: The number of books sent from distribution center j to store k.
z[i][k]: The number of books sent directly from warehouse i to store k.
------------------------------------------------------------------------
Warehouses (i):              [Tonoz(0), White_Kiosk(1), Quarterage(2), Middle_Yard(3)]
Distribution Centers (j):    [Atlas(0), Nebulae(1), Azure(2)]
Stores (k):                  [Venus(0), Minerva(1), Neptunus(2), Mars(3), Ceres(4)]


After running this code with this change to capacity value we get a new optimal transportation plan with a total cost of 95800.0. The values of decision variables x, y, and z are also different compared to their values before this change.